<a href="https://colab.research.google.com/github/schan6927/CNN/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

convet have neurons arranged in 3 dimensions : width, height, depth (depth, here, means total number of layers in network)

A ConvNet is made up of Layers. Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters.

**The fundamental reason for using CNN** is for using **less parameter** relative to the input data.

The number of parameter needed is, usually related to the number of filters, and the size of filter. (h * w * d) * n

Using the filter, we can yield a specific result we want



*   Padding


Padding is used for the edge. We can use appropriate number of padding to make the size of input and output same.



*   Stride


Stride is used for ?

여러 층을 거쳐, 필터의 크기는 줄고, 필터의 수는 늘게 된다. 그리고 최종적으로는 필터의 크기에 필터의 수에 곱한 만큼의 기다란 벡터를 가지고, softmax 나 logistic에 넣어 최종 결과를 얻게 된다.

Three types of convolutional network
*   Convolution
*   Pooling
*   Fully Connected



CNN에서 볼 수 있는 패턴


*   Conv
*   Pool
*   FC layer (Fully Connected)
*   SoftMax



CNN의 장점


*   Parameter sharing : 입력 이미지의 여러 위치에서 변수를 동일하게 사용 가능
*   Sparsity of connections : 결과값이 특정 변수(위치)에 의해서만 영향을 받음(다른 변수의 영향은 받지 않음)





# CNN Architecture

case study를 하면서, 실제로 Conv layer 를 어떻게 응용하여 모델을 만드는지 파악한다.

우선 가장 많이 사용된다는 ResNets 위주로 분석해보자

*   ResNets



# CNN 구현

pytorch 를 이용해야 함

아래의 문헌에서 코드를 따왔다. 우선 CNN이 코드로 어떻게 구현될 수 있는지, 그리고 구현되었는지를 명령 단위로 상세히 분석하기로 했다.

그 후, 직접 CNN을 구현해보는 수준까지 이르기로 한다.

* Conv layer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    """각각의 conv 층 설계"""
    # 입력을 받는 층(입력 채널 수 : 1, 출력 채널 수 : 3, 필터 크기 : 5, 스트라이드 수 : 1), 패딩은 없음
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
    # 두번 째 층(입력 채널 수 : 3, 출력 채널 수 ; 10, ), 패딩은 없음. *이전 층의 출력 채널 수와 현재 층의 입력 채널 수가 동일*
    self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
    # 완전 연결층 ?? 저 인자들이 어떻게 구체적으로 이런 수들이 나온 것인가?
    self.fc1 = nn.Linear(10 * 12 * 12, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    print("연산 전", x.size())
    # relu : conv 층 다음에 적용하는 것 (시그모이드 함수 비슷한 역할..)
    x = F.relu(self.conv1(x))
    print("conv1 연산 후", x.size())
    x = F.relu(self.conv2(x))
    print("conv2 연산 후",x.size())
    x = x.view(-1, 10 * 12 * 12)
    print("차원 감소 후", x.size())
    x = F.relu(self.fc1(x))
    print("fc1 연산 후", x.size())
    x = self.fc2(x)
    print("fc2 연산 후", x.size())
    return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))  # Input Size: (10, 1, 20, 20)

연산 전 torch.Size([10, 1, 20, 20])
conv1 연산 후 torch.Size([10, 3, 16, 16])
conv2 연산 후 torch.Size([10, 10, 12, 12])
차원 감소 후 torch.Size([10, 1440])
fc1 연산 후 torch.Size([10, 50])
fc2 연산 후 torch.Size([10, 10])




*  Max pool layer



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    """max pool 층 설계"""
    #채널의 수는 건들이지 않고, 데이터의 wieght와 height가 절반으로 줄도록 커널 사이즈를 설정함
    self.max_pool1 = nn.MaxPool2d(kernel_size=2)
    self.max_pool2 = nn.MaxPool2d(kernel_size=2)
    self.fc1 = nn.Linear(10 * 5 * 5, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    print("연산 전", x.size())
    x = F.relu(self.max_pool1(x))
    print("max_pool1 연산 후", x.size())
    x = F.relu(self.max_pool2(x))
    print("max_pool2 연산 후",x.size())
    x = x.view(-1, 10 * 5 * 5)
    print("차원 감소 후", x.size())
    x = F.relu(self.fc1(x))
    print("fc1 연산 후", x.size())
    x = self.fc2(x)
    print("fc2 연산 후", x.size())
    return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))

연산 전 torch.Size([10, 1, 20, 20])
max_pool1 연산 후 torch.Size([10, 1, 10, 10])
max_pool2 연산 후 torch.Size([10, 1, 5, 5])
차원 감소 후 torch.Size([1, 250])
fc1 연산 후 torch.Size([1, 50])
fc2 연산 후 torch.Size([1, 10])


위의 코드를 통해, 구현해낸 CNN 클래스가 실제로 layer를 거친 후, 데이터의 tensor? 가 어떻게 변화되는지를 확인할 수 있다.


MNIST 데이터셋 학습

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

데이터셋 설정

In [ ]:
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 학습 데이터
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 테스트 데이터
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)

CNN 모듈 구현

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)
#max_pool 층은 따로 밖에 구현되어 있는 이유는 생성자와 관련..?

    def forward(self, x):
        """ 하나의 층에서 일어나는 일 """
        #conv층 -> relu -> max_pool층
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 4 * 4 * 50) # [batch_size, 50, 4, 4]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

학습시키는 부분
*   10번 학습 시킨다..?
*   한번의 반복문 마다 train_loader에서 data를 가져오고 그것을 cnn에 넣는다
*   학습 결과를 criterion이라는 함수를 통해 손실을 계산하고, 해당 손실을 통해 역전파? 를 거친다
*   그 후, 최적화? 를 거친다



In [10]:
cnn.train()  # 학습을 위함
for epoch in range(2):
  for index, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()  # 기울기 초기화
    output = cnn(data)
    loss = criterion(output, target) # 오차 검출
    loss.backward()  # 역전파
    optimizer.step() # 최적화..?

    if index % 100 == 0:
      print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))

loss of 0 epoch, 0 index : 0.055339690297842026


KeyboardInterrupt: ignored

테스트시키는 부분

In [7]:
cnn.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    output = cnn(data)
    test_loss += criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 27.3272, Accuracy: 9587/10000 (96%)



In [8]:
# 모델 학습 완료 후
trained_model = CNN()

# 학습된 가중치 확인
for name, param in trained_model.state_dict().items():
    if 'conv' in name:  # 필터(커널) 가중치 확인
        print(f'{name}: {param.shape}')


conv1.weight: torch.Size([20, 1, 5, 5])
conv1.bias: torch.Size([20])
conv2.weight: torch.Size([50, 20, 5, 5])
conv2.bias: torch.Size([50])


# 실습 후 느낀 점

의문점


*   어느 모델을 이용해야 하는가? (다양한 net이 존재함)
*   구현 방법을 구체화하지 못하겠다
*   어떤 필터를 얼마나, 또 층은 얼마나 설계해야 하는 것인가? (혹시 이것이 모델의 선정과 관련되어 있는 일인가? 그렇다면 어느 모델을 선택해야 할까? 우리가 하는 작업과 최적화된 모델을 선택하는 것인가, 아니면 최신 성능의 모델을 사용하면 되는 것일까?)
  *   전반적인 모델의 설계는 층을 얼마나, 어떻게 설정하고 각각의 필터의 수를 조절하는 것이 전부인 것 같다.
*   다음 함수들의 역할
  *   criterion(output, target)
  *   loss.backward() (loss는 변수명임)
  *   optimizer.step()
*   그래서 kernel 안의 값은 무엇인가? 그 값은 어떻게 선정하는 것인가?
    *   초기에는 랜덤으로 생성되어 있고, 그 후에 역전파 알고리즘을 통해 알아서 가중치를 조절해 나간다. 따라서 직접 그 가중치를 알 필요는 아직 없는 것 같다.


    # 가중치 변화 출력
    for name, param in cnn.named_parameters():
      if param.requires_grad:
        print(f'Parameter name: {name}')
        print(f'Parameter shape: {param.shape}')
        print(f'Parameter data: {param.data}')
        print(f'Parameter gradient: {param.grad}')
        print()


개념을 다시 확실히 짚고 넘어가야할 것들
*   Parameter의 수
*   구체적으로 stride와 kernel size를 어떻게 계산해 값을 설정하는지에 대한 수식

    W’ = (W - F + 2P) / S + 1 [W': 결과값의 크기, W : 입력값의 크기, F : 필터의 크기, P : 패딩의 크기, S : 스트라이딩의 크기]

*   Fc 층의 역할, 그리고 relu의 역할에 대해서만 간단히 알고 넘어가기 (적어도 왜 쓰이는지만)

    relu 는 활성화 함수

    FC 층의 역할은

# References

https://justkode.kr/deep-learning/pytorch-cnn/
